In [ ]:
# 모든 단계 재현을 위해 파이프라인으로 구성합니다.
# 주석을 자세히 달았습니다.

import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# ----------------------------------------------------
# 1) 데이터 준비 (예시)
# ----------------------------------------------------
# df에는 최소한 다음 컬럼들이 있다고 가정합니다:
# ['Age', 'Blood_Pressure', 'Cholesterol', 'Gender_Encoded', 'target']
num_cols_for_transform = ['Blood_Pressure', 'Cholesterol']
other_cols = ['Age', 'Gender_Encoded']  # 여기서는 스케일링/변환 없이 그대로 사용 (필요시 추가 가능)

X = df[other_cols + num_cols_for_transform]
y = df['target']  # 이진/다중 라벨

# 데이터 누수를 막기 위해 반드시 'train/test split' 이후에 'fit'을 합니다.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ----------------------------------------------------
# 2) 컬럼별 전처리 정의
# ----------------------------------------------------
# (a) 콜레스테롤: log1p → MinMax
chol_pipeline = Pipeline(steps=[
    ("log1p", FunctionTransformer(np.log1p, inverse_func=np.expm1, validate=False)),
    ("scale", MinMaxScaler())
])

# (b) 혈압: Box–Cox → MinMax
#  - Box–Cox는 입력이 양수여야 합니다. 양수가 아니면 Yeo-Johnson(method='yeo-johnson')로 바꾸세요.
bp_pipeline = Pipeline(steps=[
    ("boxcox", PowerTransformer(method='box-cox', standardize=False)),
    ("scale", MinMaxScaler())
])

# (c) 나머지 수치/범주형(여기선 Age, Gender_Encoded)은 그대로 통과
#     필요하다면 별도 스케일러나 원-핫 인코더를 추가하세요.
preprocessor = ColumnTransformer(
    transformers=[
        ("bp",   bp_pipeline,   ["Blood_Pressure"]),
        ("chol", chol_pipeline, ["Cholesterol"]),
    ],
    remainder='passthrough'  # ['Age', 'Gender_Encoded']는 순서대로 뒤에 붙습니다.
)

# ColumnTransformer 출력 컬럼 순서는:
# ['Blood_Pressure(변환후)', 'Cholesterol(변환후)', 'Age(원본)', 'Gender_Encoded(원본)']

# ----------------------------------------------------
# 3) 전처리 + 분류기 파이프라인
# ----------------------------------------------------
clf = Pipeline(steps=[
    ("pre", preprocessor),
    ("svm", SVC(kernel='rbf', probability=True, random_state=42))  # 확률 출력 위해 probability=True
])

# 학습
clf.fit(X_train, y_train)

# 평가 (예시)
print("Train score:", clf.score(X_train, y_train))
print("Test  score:", clf.score(X_test, y_test))

# ----------------------------------------------------
# 4) 저장 & 로드 (전처리 파이프라인 + 모델을 통째로)
# ----------------------------------------------------
joblib.dump(clf, "svm_pipeline.pkl")

# 배포 환경/새 세션에서:
loaded = joblib.load("svm_pipeline.pkl")

# ----------------------------------------------------
# 5) 새 데이터에 동일 전처리 + 예측/신뢰도
# ----------------------------------------------------
new_data = pd.DataFrame([{
    "Age": 30,
    "Gender_Encoded": 0,
    "Blood_Pressure": 120.0,   # 반드시 학습과 같은 단위/정의
    "Cholesterol": 200.0
}])

# predict_proba로 신뢰도(예측 확률) 확인
proba = loaded.predict_proba(new_data)[0]
pred  = loaded.predict(new_data)[0]
classes = loaded.named_steps["svm"].classes_

print("예측 라벨:", pred)
print("클래스별 확률:")
for c, p in zip(classes, proba):
    print(f"  {c}: {p:.4f}")
print("신뢰도(예측 라벨 확률):", float(np.max(proba)))
